In [ ]:
import tensorflow as tf
import logging
from autologging import logged, traced, TRACE
logging.basicConfig(level=logging.INFO, format="%(name)s.%(funcName)s\t%(message)s")
logging.info(f"tensorflow version {tf.version.VERSION}")

In [ ]:
import numpy as np
from zebrastack_model_v2 import prepare_images

from tensorflow import keras
(train_images, _), (test_images, _) = keras.datasets.fashion_mnist.load_data()
train_images, test_images = train_images[:len(train_images)//1], test_images[:len(test_images)//1]
logging.info(f"train_images: {train_images.shape} {train_images.dtype}")

train_images = prepare_images(train_images)
test_images = prepare_images(test_images)

In [ ]:
from zebrastack_model_v2 import ZebraStackModel

model = ZebraStackModel(latent_dim=8)
model.encoder.summary()
model.decoder.summary()

In [ ]:
from figure_callback import FigureCallback

nb_callback = FigureCallback(fig)

model.train(train_images, test_images, nb_callback)